# Workspace resources

<table align="left">

  <td>
    <a href="https://github.com/DataBiosphere/terra-axon-examples/blob/main/first_hour_on_terra/workspace_resources.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/DataBiosphere/terra-axon-examples/main/first_hour_on_terra/workspace_resources.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in a Terra notebook instance
    </a>
  </td>                                                                                               
</table>

## Overview

This notebook provides examples of working with workspace resources. Build upon the best practices described in this notebook to include and share resources in your own workspaces. 

### Objective

Perform common workspace resource operations including:

1. Viewing the resources available in an existing workspace.
1. Creating new resources in an existing workspace.
1. Viewing data from a cloud resource in your Terra cloud environment.
#### How to run this notebook

Run the setup section before running the cells of the other sections.

#### Costs

This notebook takes less than a minute to run, which will typically cost less than $0.01 of compute time on your cloud environment.

### Notebook setup

#### Set up utility functions

In [ ]:
import json

'''
Resolves bucket URL from bucket reference in workspace.
'''
def get_bucket_url_from_reference(bucket_reference):
    BUCKET_CMD_OUTPUT = !terra resolve --name={bucket_reference}
    BUCKET = BUCKET_CMD_OUTPUT[0]
    return BUCKET

'''
Resolves current workspace ID from workspace description.
'''
def get_current_workspace_id():
    WORKSPACE_CMD_OUTPUT = !terra workspace describe --format=json | jq --raw-output ".id"
    WORKSPACE_ID = WORKSPACE_CMD_OUTPUT[0]
    return WORKSPACE_ID

CURRENT_WORKSPACE_ID = get_current_workspace_id()

In [ ]:
# Widget utilities
import subprocess
import ipywidgets as widgets
from IPython.display import display,HTML

input_style= {
    'description_width':'initial'
}

input_layout = widgets.Layout(
    width='400px',
    justify_content='flex-start'
)

# cloning_drop_down = widgets.Dropdown(options=['COPY_NOTHING','COPY_DEFINITION','COPY_RESOURCE','COPY_REFERENCE'],
#                                    value='COPY_REFERENCE',
#                                    disabled=False,
#                                    description="Cloning:")

# access_drop_down = widgets.Dropdown(
#     options=['SHARED_ACCESS','PRIVATE_ACCESS'],
#     value='SHARED_ACCESS',
#     disabled=False,
#     description="Access:")

# gcp_storage_link = widgets.HTML(
#     value="<a href='https://cloud.google.com/storage/docs/storage-classes'>Storage Class</a>",
# )

storageDropDown = widgets.Dropdown(options=['STANDARD','NEARLINE','COLDLINE','ARCHIVE'],
                                   value='STANDARD',
                                   style=input_style,
                                   description="Storage Class:")

#### Workspace setup
<div class="alert alert-block alert-info">
<b>Note:</b> This notebook assumes that <a href="../../terra-axon-examples/workspace_setup.ipynb">`workspace_setup.ipynb`</a> has been run.
</div>
    
`workspace_setup.ipynb` creates two Cloud Storage buckets for your workspace files with workspace reference names: 

 - ws_files   
 - ws_files_autodelete_after_two_weeks      
    
The code in this notebook will write output files to the "autodelete" bucket by default.  
    Any file in this bucket will be automatically deleted <b>two weeks</b> after it is written.  
    This alleviates the need for you to remember to clean up temporary and example files manually.  
    If you want to write outputs to a durable location, simply change the assignment of the `BUCKET_REFERENCE` variable in the cell below and re-run the notebook. 

In [ ]:
# Change this to "ws_files" to use the durable workspace bucket instead of the autodelete bucket.
BUCKET_REFERENCE = "ws_files_autodelete_after_two_weeks"

In [ ]:
MY_BUCKET = get_bucket_url_from_reference(BUCKET_REFERENCE)
print(f'Bucket ID: {MY_BUCKET}')

### Add referenced resources

[Referenced resources](https://terra-docs.api.verily.com/docs/getting_started/web_ui/#referenced-vs-workspace-controlled-resources) represent data and other elements in Enterprise Terra by pointing to a source that exists outside of the current workspace.

For each type of referenced resource supported by Enterprise Terra, this notebook provides
1. A **widget** which runs a Terra CLI command to add a referenced resource.<br>simply fill in the inputs with your desired values and click the button.
1. A **video** showing how to create that type of resource in the Enterprise Terra UI.



#### 1. Add a GitHub repository

Both public and private GitHub repositories can be added to data collections. In order to add a private GitHub repository, you must first [set up your Terra SSH key](https://terra-docs.api.verily.com/docs/how_to_guides/terra_ssh_key_guide) and should provide a repo URL of the format `git@github.com` instead of `http://`. No additional setup is required to add a public GitHub repository.

Once you've run the cell below, populated the input fields and clicked the button, the output should resemble:
```
Successfully added referenced git repo.
Name:         dc-public-repo
Description:  A public GitHub repo
Type:         GIT_REPO
Stewardship:  REFERENCED
Cloning:      COPY_REFERENCE
Properties:   class Properties {
    []
}
Git repo Url: https://github.com/<YOUR_REPO>
```

In [ ]:
class AddRepoWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide appropriate values in the input boxes.')
        self.input_name = widgets.Text(
        placeholder="<REPO_NAME>",
        description="Name:"
        )
        self.input_description = widgets.Text(
            placeholder="<REPO_DESCRIPTION>",
            description="Description:"
        )
        self.input_repo_url = widgets.Text(
            placeholder="<REPO_URL>",
            description="URL:"
        )
        self.cloning_drop_down = widgets.Dropdown(options=['COPY_NOTHING','COPY_DEFINITION','COPY_RESOURCE','COPY_REFERENCE'],
                                                value='COPY_REFERENCE',disabled=False,description="Cloning:")
        self.output = widgets.Output()
        self.add_repo_button = widgets.Button(
            description='Add resource',
            disabled=False,
            button_style='',
            tooltip='abcd',
            icon='plus'
        )
        self.add_repo_button.on_click(self.add_repo)
        self.vb = widgets.VBox(
            [
                self.label,
                self.cloning_drop_down,
                self.input_name,
                self.input_description,
                self.input_repo_url,
                self.add_repo_button,
                self.output
            ]
        )
    
    def add_repo(self,b):
        with self.output:
            # Cast description input to string.
            description_content = "\"{0}\"".format(self.input_description.value)
            print("Running command to add repo...")
            terraCommand = f"""terra resource add-ref git-repo \\
            --cloning={self.cloning_drop_down.value} \\
            --description={description_content} \\
            --name={self.input_name.value} \\
            --repo-url={self.input_repo_url.value} \\
            --workspace={CURRENT_WORKSPACE_ID}"""
            print(terraCommand)
            result = subprocess.run(["terra","resource","add-ref","git-repo",
                                     f"--cloning={cloning_drop_down.value}",
                                     f"--description={description_content}",
                                     f"--name={input_name.value}",
                                     f"--repo-url={input_repo_url.value}",
                                     f"--workspace={CURRENT_WORKSPACE_ID}"],
                                    capture_output=True,
                                    text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)
            
# Instantiate widget
add_repo_widget = AddRepoWidget()
display(add_repo_widget.vb)

#### 2. Add a BigQuery dataset

This section contains a widget that enables you to add a BigQuery dataset as a referenced resource to your workspace.

Please note that you must populate either the path field (which will have format `<PROJECT_ID>.<DATASET_ID>`) field or the both [dataset ID and the project ID](https://cloud.google.com/bigquery/docs/datasets-intro#datasets) fields for this widget to run successfully.

Once you've run the cell below, populated the input fields and clicked the button, the output should resemble:
```
Successfully added referenced BigQuery dataset.
Name:         <NAME>
Description:  <DESCRIPTION>
Type:         BQ_DATASET
Stewardship:  REFERENCED
Cloning:      COPY_REFERENCE
Properties:   class Properties {
    []
}
GCP project id: <PROJECT_ID>
BigQuery dataset id: <DATASET_ID>
Location: US
# Tables: <NUMBER_OF_TABLES>
```

In [ ]:
class AddBQDatasetWidget(object):
    def __init__(self):
        self.label = widgets.Label(
            value="Please provide appropriate values in the input boxes."
        )
        self.cloning_drop_down = widgets.Dropdown(
            options=[
                "COPY_NOTHING",
                "COPY_DEFINITION",
                "COPY_RESOURCE",
                "COPY_REFERENCE",
            ],
            value="COPY_REFERENCE",
            disabled=False,
            description="Cloning:",
        )
        self.input_dataset_id = widgets.Text(
            placeholder="<DATASET_ID>", description="Dataset ID:"
        )
        self.input_description = widgets.Text(
            placeholder="<REPO_DESCRIPTION>", description="Description:"
        )
        self.input_name = widgets.Text(placeholder="<NAME>", description="Name:")
        self.input_path = widgets.Text(placeholder="<PATH>", description="Path:")
        self.input_project_id = widgets.Text(
            placeholder="<GCP_PROJECT_ID>",
            description="GCP Project ID:",
            style=input_style,
        )
        self.output = widgets.Output()
        self.button = widgets.Button(
            description="Add resource",
            disabled=False,
            button_style="",
            tooltip="Click to add a BigQuery dataset as a referenced resource",
            icon="plus",
        )
        self.button.on_click(self.add_dataset)
        self.vb = widgets.VBox(
            [
                self.label,
                self.cloning_drop_down,
                self.input_dataset_id,
                self.input_description,
                self.input_name,
                self.input_path,
                self.input_project_id,
                self.button,
                self.output
            ]
        )
    def add_dataset(self, b):
        with self.output:
            # Cast description to string.
            description_content = '"{0}"'.format(self.input_description.value)

            terraCommand = f"""terra resource add-ref bq-dataset \\
            --cloning={self.cloning_drop_down.value} \\
            --dataset_id={self.input_dataset_id.value} \\
            --description={description_content} \\
            --name={self.input_name.value} \\
            --path={self.input_path.value} \\
            --project_id={self.input_project_id.value} \\
            --workspace={CURRENT_WORKSPACE_ID}
            """
            print(terraCommand)

            if not self.input_path.value:
                result = subprocess.run(
                    [
                        "terra",
                        "resource",
                        "add-ref",
                        "bq-dataset",
                        f"--cloning={self.cloning_drop_down.value}",
                        f"--dataset-id={self.input_dataset_id.value}",
                        f"--description={description_content}",
                        f"--name={self.input_name.value}",
                        f"--project-id={self.input_project_id.value}",
                        f"--workspace={CURRENT_WORKSPACE_ID}",
                    ],
                    capture_output=True,
                    text=True,
                )
            else:
                result = subprocess.run(
                    [
                        "terra",
                        "resource",
                        "add-ref",
                        "bq-dataset",
                        f"--cloning={self.cloning_drop_down.value}",
                        f"--description={description_content}",
                        f"--name={self.input_name.value}",
                        f"--path={self.input_path.value}",
                        f"--workspace={CURRENT_WORKSPACE_ID}",
                    ],
                    capture_output=True,
                    text=True,
                )
        print(result.stderr) if not result.stdout else print(result.stdout)


# Instantiate widget
add_bq_dataset_widget = AddBQDatasetWidget()
display(add_bq_dataset_widget.vb)

#### 3. Add a BigQuery table

The cell below creates a widget that adds a BigQuery dataset as a referenced resource to your workspace.

Please note that you can choose to populate either 
- the path field (which will have format `<PROJECT_ID>.<DATASET_ID>.<TABLE_ID>`) field, or
- the dataset ID, project ID and [table ID](https://cloud.google.com/bigquery/docs/datasets-intro#datasets) fields.

Once you've run the cell below, populated the input fields and clicked the button, the output should resemble:
```
Successfully added referenced BigQuery data table.
Name:         <NAME>
Description:  <DESCRIPTION>
Type:         BQ_TABLE
Stewardship:  REFERENCED
Cloning:      COPY_REFERENCE
Properties:   class Properties {
    []
}
GCP project id: <PROJECT_ID>
BigQuery dataset id: <DATASET_ID>
BigQuery table id: <TABLE_ID>
# Rows: (unknown)
```

In [ ]:
class AddBQTableWidget(object):

    def __init__(self):
        self.cloning_drop_down = widgets.Dropdown(
            options=[
                "COPY_NOTHING",
                "COPY_DEFINITION",
                "COPY_RESOURCE",
                "COPY_REFERENCE",
            ],
            value="COPY_REFERENCE",
            disabled=False,
            description="Cloning:",
        )
        self.input_dataset_id = widgets.Text(
            placeholder="<DATASET_ID>", description="Dataset ID:"
        )
        self.input_description = widgets.Text(
            placeholder="<REPO_DESCRIPTION>", description="Description:"
        )
        self.input_name = widgets.Text(
            placeholder="<NAME>", description="Name:")
        self.input_path = widgets.Text(
            placeholder="<PATH>", description="Path:")
        self.input_project_id = widgets.Text(
            placeholder="<GCP_PROJECT_ID>",
            description="GCP Project ID:",
            style=input_style,
        )
        self.input_table_id = widgets.Text(
            placeholder="<TABLE_ID>", description="Table ID:", style=input_style
        )
        self.output = widgets.Output()
        self.button = widgets.Button(
            description="Add resource",
            disabled=False,
            button_style="",
            tooltip="Click to add a BigQuery table as a referenced resource.",
            icon="plus",
        )
        self.button.on_click(self.add_bq_table)
        self.vb = widgets.VBox(
            [
                self.cloning_drop_down,
                self.input_dataset_id,
                self.input_description,
                self.input_name,
                self.input_path,
                self.input_project_id,
                self.input_table_id,
                self.button,
                self.output
            ]
        )

    def add_bq_table(self, b):
        with self.output:
            
            description_content = f"{self.input_description.value}"

            terraCommand = f"""terra resource add-ref bq-table \\
            --cloning={self.cloning_drop_down.value} \\
            --dataset_id={self.input_dataset_id.value} \\
            --description={description_content} \\
            --name={self.input_name.value} \\
            --path={self.input_path.value} \\
            --project_id={self.input_project_id.value} \\
            --table_id={self.input_table_id.value} \\
            --workspace={CURRENT_WORKSPACE_ID}
            """
            print(terraCommand)

            if not self.input_path.value:
                result = subprocess.run(
                    [
                        "terra",
                        "resource",
                        "add-ref",
                        "bq-table",
                        f"--cloning={self.cloning_drop_down.value}",
                        f"--dataset-id={self.input_dataset_id.value}",
                        f"--description={description_content}",
                        f"--name={self.input_name.value}",
                        f"--project-id={self.input_project_id.value}",
                        f"--table-id={self.input_table_id.value}",
                        f"--workspace={CURRENT_WORKSPACE_ID}",
                    ],
                    capture_output=True,
                    text=True,
                )
            else:
                result = subprocess.run(
                    [
                        "terra",
                        "resource",
                        "add-ref",
                        "bq-table",
                        f"--cloning={self.cloning_drop_down.value}",
                        f"--description={self.description_content}",
                        f"--name={self.input_name.value}",
                        f"--path={self.input_path.value}",
                        f"--workspace={CURRENT_WORKSPACE_ID}",
                    ],
                    capture_output=True,
                    text=True,
                )
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
add_bq_table_widget = AddBQTableWidget()
display(add_bq_table_widget.vb)

#### 4. Add a Google Cloud Storage bucket

This section contains a widget which adds a Google Cloud Storage bucket as a referenced resource to your workspace.<br>Once you've run the cell below, populated the input fields and clicked the button, the output should resemble:
```
Successfully added referenced GCS bucket.
Name:         <NAME>
Description:  <DESCRIPTION>
Type:         GCS_BUCKET
Stewardship:  REFERENCED
Cloning:      COPY_REFERENCE
Properties:   class Properties {
    []
}
GCS bucket name: <BUCKET_NAME>
Location: US-CENTRAL1
# Objects: <NUMBER_OF_OBJECTS>
```

In [ ]:
class AddGcsBucketWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values in the input boxes.')
        self.cloning_drop_down = widgets.Dropdown(
            options=[
                "COPY_NOTHING",
                "COPY_DEFINITION",
                "COPY_RESOURCE",
                "COPY_REFERENCE",
            ],
            value="COPY_REFERENCE",
            disabled=False,
            description="Cloning:",
        )
        self.input_bucket_name = widgets.Text(
         placeholder="<BUCKET_NAME>",
         description="Bucket Name:",
         style=input_style
         )
        self.input_description = widgets.Text(
         placeholder="<DESCRIPTION>",
         description="Description:",
         style=input_style
         )
        self.input_name = widgets.Text(
         placeholder="<NAME>",
         description="Name:",
         style=input_style
         )
        self.output = widgets.Output()
        self.button = widgets.Button(
            description='Add resource',
            disabled=False,
            button_style='',
            tooltip='Click to add a GCS bucket as a referenced resource.',
            icon='plus'
        )
        self.vb = widgets.VBox(
            [
                self.label,
                self.cloning_drop_down,
                self.input_bucket_name,
                self.input_description,
                self.input_name,
                self.button,
                self.output
            ]
        )
        self.button.on_click(self.add_gcs_bucket)
        
    def add_gcs_bucket(self,b):
        with self.output:
            description_content = f"\"{self.input_description.value}\""

            terraCommand = f"""terra resource add-ref gcs-bucket \\
            --cloning={self.cloning_drop_down.value} \\
            --bucket-name={self.input_bucket_name.value} \\
            --description={description_content} \\
            --name={self.input_name.value} \\
            --workspace={CURRENT_WORKSPACE_ID}
            """
            print(terraCommand)
            result = subprocess.run(["terra","resource","add-ref","gcs-bucket",
                             f"--cloning={self.cloning_drop_down.value}",
                             f"--bucket-name={self.input_bucket_name.value}",
                             f"--name={self.input_name.value}",
                             f"--description={description_content}",
                             f"--workspace={CURRENT_WORKSPACE_ID}"],capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)
            

# Instantiate widget
add_gcs_bucket_widget = AddGcsBucketWidget()
display(add_gcs_bucket_widget.vb)

#### 5. Add Google Cloud Storage object

This section contains a widget that adds a BigQuery dataset as a referenced resource to your workspace.

Please note that you can choose to populate either 
- the bucket name and path fields, or
- the bucket name and object name fields, wherein the object name is the full path to the object (e.g. `/folder1/folder2/OBJECT.txt`).

Once you've run the cell below, populated the input fields and clicked the button, the output should resemble:

In [ ]:
class AddGcsObjectWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values in the input boxes.')
        self.cloning_drop_down = widgets.Dropdown(
            options=[
                "COPY_NOTHING",
                "COPY_DEFINITION",
                "COPY_RESOURCE",
                "COPY_REFERENCE",
            ],
            value="COPY_REFERENCE",
            disabled=False,
            description="Cloning:",
        )
        self.input_bucket_name = widgets.Text(
         placeholder="<BUCKET_NAME>",
         description="Bucket Name:",
         style=input_style
         )
        self.input_description = widgets.Text(
         placeholder="<DESCRIPTION>",
         description="Description:"
         )
        self.input_name = widgets.Text(
         placeholder="<NAME>",
         description="Name:"
         )
        self.input_object_name = widgets.Text(
         placeholder="<OBJECT_NAME>",
         description="Object Name:",
         style=input_style
         )
        self.input_path = widgets.Text(
         placeholder="<PATH>",
         description="Path to Object:",
         style=input_style
         )
        self.output = widgets.Output()
        self.button = widgets.Button(
            description='Add resource',
            disabled=False,
            button_style='',
            tooltip='Click to add a GCS bucket as a referenced resource.',
            icon='plus'
        )
        self.button.on_click(self.add_gcs_object)
        self.vb = widgets.VBox([
            self.label,
            self.cloning_drop_down,
            self.input_bucket_name,
            self.input_description,
            self.input_name,
            self.input_object_name,
            self.input_path,
            self.button,
            self.output])

    # define a function for the button to call
    def add_gcs_object(self,b):
        with self.output:
            # Cast description input to string.
            description_content = f"\"{self.input_description.value}\""

            if not self.input_path.value:
                terraCommand = f"""terra resource add-ref gcs-object \\
                --cloning={self.cloning_drop_down.value} \\
                --bucket-name={self.input_bucket_name.value} \\
                --description={description_content} \\
                --name={self.input_name.value} \\
                --object-name={self.input_object_name.value} \\
                --workspace={CURRENT_WORKSPACE_ID}"""
                print(terraCommand)
                result = subprocess.run(["terra","resource","add-ref","gcs-object",
                                         f"--cloning={self.cloning_drop_down.value}",
                                         f"--bucket-name={self.input_bucket_name.value}",
                                         f"--description={description_content}",
                                         f"--name={self.input_name.value}",
                                         f"--object-name={self.input_object_name.value}",
                                         f"--workspace={CURRENT_WORKSPACE_ID}"],capture_output=True,text=True)
                print(result.stderr) if not result.stdout else print(result.stdout)
            else:
                print("Path provided")
                terraCommand = f"""terra resource add-ref gcs-object \\
                --cloning={self.cloning_drop_down.value} \\
                --bucket-name={self.input_bucket_name.value} \\
                --description={description_content} \\
                --name={self.input_name.value} \\
                --path={self.input_path.value} \\
                --workspace={CURRENT_WORKSPACE_ID}"""
                print(terraCommand)
                result = subprocess.run(["terra","resource","add-ref","gcs-bucket",
                                         f"--cloning={self.cloning_drop_down.value}",
                                         f"--bucket-name={self.input_bucket_name.value}",
                                         f"--description={description_content}",
                                         f"--name={self.input_name.value}",
                                         f"--path={self.input_path.value}",
                                         f"--workspace={CURRENT_WORKSPACE_ID}"],capture_output=True,text=True)
                print(result.stderr) if not result.stdout else print(result.stdout)

add_gcs_obj_widget = AddGcsObjectWidget()
display(add_gcs_obj_widget.vb)

### Add controlled resources

[Controlled resources](https://terra-docs.api.verily.com/docs/getting_started/web_ui/#referenced-vs-workspace-controlled-resources) are cloud resources that are managed or created by Enterprise Terra within the current workspace. 
For each type of controlled resource supported in Enterprise Terra, this notebook provides:

1. A widget which runs a Terra CLI command to add a referenced resource.
simply fill in the inputs with your desired values and click the button.
1. A video showing how to create that type of resource in the Enterprise Terra UI.

#### 1. Create a BigQuery dataset

Using the widget in this section, you can create a BigQuery dataset as a controlled resource of this workspace. 
Note that if a dataset ID is not provided, the resource name will be used for dataset ID. 

Once you've run the cell below, populated the input fields and clicked the button, the output should resemble:
```
Successfully added controlled BigQuery dataset.
Name:         <NAME>
Description:  <DESCRIPTION>
Type:         BQ_DATASET
Stewardship:  CONTROLLED
Cloning:      COPY_REFERENCE
Access scope: SHARED_ACCESS
Managed by:   USER
Properties:   class Properties {
    []
}
GCP project id: <GOOGLE_PROJECT_ID>
BigQuery dataset id: <DATASET_ID>
Location: us-central1
# Tables: 0
```

In [ ]:
class CreateBQDataset(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values in the input boxes.')
        self.cloning_drop_down = widgets.Dropdown(
            options=[
                "COPY_NOTHING",
                "COPY_DEFINITION",
                "COPY_RESOURCE",
                "COPY_REFERENCE",
            ],
            value="COPY_REFERENCE",
            disabled=False,
            description="Cloning:",
        )
        self.access_drop_down =  widgets.Dropdown(
            options=['SHARED_ACCESS','PRIVATE_ACCESS'],
            value='SHARED_ACCESS',
            disabled=False,
            description="Access:"
        )
        self.input_dataset_id = widgets.Text(
         placeholder="<DATASET ID>",
         description="Dataset ID",
         style=input_style
         )

        self.input_description = widgets.Text(
         placeholder="<DESCRIPTION>",
         description="Description:"
         )
        self.input_name = widgets.Text(
         placeholder="<NAME>",
         description="Name:"
         )
        self.output = widgets.Output()
        self.button = widgets.Button(
            description='Add resource',
            disabled=False,
            button_style='',
            tooltip='Click to add a BigQuery dataset as a controlled resource.',
            icon='plus'
        )
        self.vb = widgets.VBox([
            self.label,
            self.access_drop_down,
            self.cloning_drop_down,
            self.input_dataset_id,
            self.input_description,
            self.input_name,
            self.button,
            self.output
        ])
        self.button.on_click(self.create_bq_dataset)


    def create_bq_dataset(self,b):
        with self.output:
            description_content = f"\"{self.input_description.value}\""

            if not self.input_dataset_id.value:
                terraCommand = f"""terra resource create bq-dataset \\
                --access={self.access_drop_down.value} \\
                --cloning={self.cloning_drop_down.value} \\
                --dataset-id={self.input_dataset_id.value} \\
                --description={description_content} \\
                --name={self.input_name.value} \\
                --workspace={CURRENT_WORKSPACE_ID}"""
                result = subprocess.run(["terra","resource","create","bq-dataset",
                                         f"--access={self.access_drop_down.value}",
                                         f"--cloning={self.cloning_drop_down.value}",
                                         f"--dataset-id={self.input_dataset_id.value}",
                                         f"--description={description_content}",
                                         f"--name={self.input_name.value}",
                                         f"--workspace={CURRENT_WORKSPACE_ID}"],
                                        capture_output=True,
                                        text=True)
                print(result.stderr) if not result.stdout else print(result.stdout)
            else:            
                terraCommand = f"""terra resource create bq-dataset \\
                --access={self.access_drop_down.value} \\
                --cloning={self.cloning_drop_down.value} \\
                --description={description_content} \\
                --name={self.input_name.value} \\
                --workspace={CURRENT_WORKSPACE_ID}"""
                print(terraCommand)
                result = subprocess.run(["terra","resource","create","bq-dataset",
                                         f"--access={self.access_drop_down.value}",
                                         f"--cloning={self.cloning_drop_down.value}",
                                         f"--description={description_content}",
                                         f"--name={self.input_name.value}",
                                         f"--workspace={CURRENT_WORKSPACE_ID}"],
                                        capture_output=True,
                                        text=True)
                print(result.stderr) if not result.stdout else print(result.stdout)

create_bq_dataset_widget = CreateBQDataset()
display(create_bq_dataset_widget.vb)

#### 2. Create a Google Cloud Storage bucket

Using the widget in this section, you can create a GCS bucket as a controlled resource of this workspace. 
Note that if no bucket name is provided, one will be provided by combining the resource name and the Google project ID.
Once you've run the cell below, populated the input fields and clicked the button, the output should resemble:
```
Successfully added controlled GCS bucket.
Name:         <NAME>
Description:  <DESCRIPTION>
Type:         GCS_BUCKET
Stewardship:  CONTROLLED
Cloning:      <CLONING>
Access scope: <ACCESS>
Managed by:   USER
Properties:   class Properties {
    []
}
GCS bucket name: <BUCKET_NAME>
Location: US-CENTRAL1
# Objects: 0
```

In [ ]:
class CreateGcsBucketWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values in the input boxes.')
        self.access_drop_down = widgets.Dropdown(
            options=['SHARED_ACCESS','PRIVATE_ACCESS'],
            value='SHARED_ACCESS',
            disabled=False,
            description="Access:"
        )
        self.input_auto_delete = widgets.Text(
         placeholder="<AUTODELETE (DAYS)>",
         description="Auto Delete (Days):",
         style=input_style
        )
        self.input_bucket_name = widgets.Text(
         placeholder="<BUCKET_NAME>",
         description="Bucket Name:",
         style=input_style
         )
        self.cloning_drop_down = widgets.Dropdown(
            options=[
                "COPY_NOTHING",
                "COPY_DEFINITION",
                "COPY_RESOURCE",
                "COPY_REFERENCE",
            ],
            value="COPY_REFERENCE",
            disabled=False,
            description="Cloning:",
        )
        self.input_description = widgets.Text(
         placeholder="<DESCRIPTION>",
         description="Description:"
         )
        self.input_name = widgets.Text(
         placeholder="<NAME>",
         description="Name:"
         )
        self.input_lifecycle = widgets.Text(
         placeholder = "<PATH_TO_JSON>",
         description = "Lifecycle (JSON)",
         style=input_style
        )
        self.input_object_name = widgets.Text(
         placeholder="<OBJECT_NAME>",
         description="Object Name:",
         style=input_style
         )
        self.storage_drop_down = widgets.Dropdown(options=['STANDARD','NEARLINE','COLDLINE','ARCHIVE'],
                                   value='STANDARD',
                                   style=input_style,
                                   description="Storage Class:")
        self.input_path = widgets.Text(
         placeholder="<PATH>",
         description="Path to Object:",
         style=input_style
         )
        self.output = widgets.Output()
        self.button = widgets.Button(
            description='Add resource',
            disabled=False,
            button_style='',
            tooltip='Click to add a BigQuery dataset as a controlled resource.',
            icon='plus'
        )
        self.button.on_click(self.create_gcs_bucket)
        self.vb = widgets.VBox(
            [
                self.label,
                self.access_drop_down,
                self.input_auto_delete,
                self.input_bucket_name,
                self.cloning_drop_down,
                self.input_description,
                self.input_lifecycle,
                self.storage_drop_down,
                self.input_name,
            ]
        )

    def create_gcs_bucket(self,b):
        with self.output:
            description_content = f"\"{self.input_description.value}\""

            terraCommand = f"""terra resource create gcs-bucket \\
                --access={self.access_drop_down.value} \\
                --auto-delete={self.input_auto_delet.value} \\
                --bucket-name={self.input_bucket_name.value} \\
                --cloning={self.cloning_drop_down.value} \\
                --description={description_content} \\
                --name={self.input_name.value} \\
                --storage={self.storage_drop_down.value} \\
                --workspace={CURRENT_WORKSPACE_ID}"""     
            print(terraCommand)
            result = subprocess.run(["terra","resource","create","gcs-bucket",
                                     f"--access={self.access_drop_down.value}",
                                     f"--auto-delete={self.input_auto_delete.value}",
                                     f"--bucket-name={self.input_bucket_name.value}",
                                     f"--cloning={self.cloning_drop_down.value}",
                                     f"--description={description_content}",
                                     f"--name={self.input_name.value}",
                                     f"--storage={self.storageDropDown.value}",
                                     f"--workspace={CURRENT_WORKSPACE_ID}"],capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)


create_gcs_bucket_widget = CreateGcsBucketWidget()
display(create_gcs_bucket_widget.vb)

#### 3. Create a GCP notebook

Using the widget in this section, you can create a GCP notebook as a controlled resource of this workspace. 
If no post-startup script is provided, the [default script](https://raw.githubusercontent.com/DataBiosphere/terra-workspace-manager/main/service/src/main/java/bio/terra/workspace/service/resource/controlled/cloud/gcp/ainotebook/post-startup.sh) will be used, which is recommended in most cases.
Once you've run the cell below, populated the input fields and clicked the button, the output should resemble:
```
Successfully added controlled GCP Notebook instance.
Name:         <NAME>
Description:  <DESCRIPTION>
Type:         AI_NOTEBOOK
Stewardship:  CONTROLLED
Cloning:      COPY_NOTHING
Access scope: PRIVATE_ACCESS
Managed by:   USER
Region:       us-central1
Private user: <TERRA_USER_EMAIL>
Properties:   class Properties {
    []
}
GCP project id: <GOOGLE_PROJECT_ID>
Instance id:   <INSTANCE_ID>
Location: us-central1-a
Instance name: projects/<GOOGLE_PROJECT_ID>/locations/us-central1-a/instances/<INSTANCE_ID>
State:         PROVISIONING
Metadata:
   notebooks-api-version: v1
   post-startup-script: https://raw.githubusercontent.com/DataBiosphere/terra-workspace-manager/main/service/src/main/java/bio/terra/workspace/service/resource/controlled/cloud/gcp/ainotebook/post-startup.sh
   disable-swap-binaries: true
   serial-port-logging-enable: true
   terra-cli-server: verily-preprod
   terra-workspace-id: first-hour-on-vwb-ws
   proxy-mode: service_account
   enable-guest-attributes: TRUE
   warmup-libraries: matplotlib.pyplot
   shutdown-script: /opt/deeplearning/bin/shutdown_script.sh
   notebooks-api: PROD
Proxy URL:     (undefined)
Create time:   <YYYY-MM-DDTHH:MM:SS>
```

In [ ]:
class CreateGcpNotebookWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values in the input boxes.')
        self.access_drop_down = widgets.Dropdown(
            options=['SHARED_ACCESS','PRIVATE_ACCESS'],
            value='SHARED_ACCESS',
            disabled=False,
            description="Access:"
        )
        self.cloning_drop_down = widgets.Dropdown(
            options=[
                "COPY_NOTHING",
                "COPY_DEFINITION",
                "COPY_RESOURCE",
                "COPY_REFERENCE",
            ],
            value="COPY_REFERENCE",
            disabled=False,
            description="Cloning:",
        )
        self.input_name = widgets.Text(
         placeholder="<NAME>",
         description="Name:",
         style=input_style,
            layout=input_layout
         )
        self.input_description = widgets.Text(
         placeholder="<DESCRIPTION>",
         description="Description:",
         style=input_style,
         layout=input_layout
         )
        self.input_instance_id = widgets.Text(
         placeholder = "<INSTANCE_ID>",
         description = "Instance ID (Cannot Be Changed):",
         style=input_style,
         layout=input_layout
        )
        self.input_post_startup_script = widgets.Text(
         placeholder="<PATH TO SCRIPT (gs://)>",
         description="Post-Startup Script:",
         style=input_style,
         layout=input_layout
        )
        self.output = widgets.Output()
        self.button = widgets.Button(
            description='Add resource',
            disabled=False,
            button_style='',
            tooltip='Click to add a BigQuery dataset as a controlled resource.',
            icon='plus'
        )
        self.button.on_click(self.create_gcp_notebook)
        self.vb = widgets.VBox(
            [
                self.label,
                self.access_drop_down,
                self.cloning_drop_down,
                self.input_name,
                self.input_description,
                self.input_instance_id,
                self.input_post_startup_script,
                self.button,
                self.output
            ]
        )
        
    def create_gcp_notebook(self,b):
        with self.output:
            description_content = f"\"{self.input_description.value}\""

            if not self.input_post_startup_script.value:
                terraCommand = f"""terra resource create gcp-notebook \\
                    --access={self.access_drop_down.value} \\
                    --cloning={self.cloning_drop_down.value} \\
                    --name={self.input_name.value} \\
                    --description={description_content} \\
                    --instance-id={self.input_instance_id.value} \\
                    --workspace={CURRENT_WORKSPACE_ID}"""       
                print(terraCommand)
                result = subprocess.run(["terra","resource","create","gcp-notebook",
                                         f"--access={self.access_drop_down.value}",
                                         f"--cloning={self.cloning_drop_down.value}",
                                         f"--name={self.input_name.value}",
                                         f"--description={description_content}",
                                         f"--instance-id={self.input_instance_id.value}",
                                         f"--workspace={CURRENT_WORKSPACE_ID}"],capture_output=True,text=True)
                print(result.stderr) if not result.stdout else print(result.stdout)
            else:
                terraCommand = f"""terra resource create gcp-notebook \\
                    --access={self.access_drop_down.value} \\
                    --cloning={self.cloning_drop_down.value} \\
                    --name={self.input_name.value} \\
                    --description={description_content} \\
                    --instance-id={self.input_instance_id.value} \\
                    --post-startup-script={self.input_post_startup_script.value} \\
                    --workspace={CURRENT_WORKSPACE_ID}"""            
                print(terraCommand)
                result = subprocess.run(["terra","resource","create","gcp-notebook",
                                         f"--access={self.access_drop_down.value}",
                                         f"--cloning={self.cloning_drop_down.value}",
                                         f"--name={self.input_name.value}",
                                         f"--description={description_content}",
                                         f"--instance-id={self.input_instance_id.value}",
                                         f"--post-startup-script={self.input_post_startup_script.value}",
                                         f"--workspace={CURRENT_WORKSPACE_ID}"],capture_output=True,text=True)
                print(result.stderr) if not result.stdout else print(result.stdout)

create_gcp_notebook_widget = CreateGcpNotebookWidget()
display(create_gcp_notebook_widget.vb)

## Provenance

Generate information about this notebook environment and the packages installed.

In [ ]:
!date

Conda and pip installed packages:

In [ ]:
!conda env export

JupyterLab extensions:

In [ ]:
!jupyter labextension list

Number of cores:

In [ ]:
!grep ^processor /proc/cpuinfo | wc -l

Memory:

In [ ]:
!grep "^MemTotal:" /proc/meminfo

---
Copyright 2022 Verily Life Sciences LLC

Use of this source code is governed by a BSD-style   
license that can be found in the LICENSE file or at   
https://developers.google.com/open-source/licenses/bsd